In [ ]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [1]:
import numpy as np
import pandas as pd
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf
from tensorflow import keras

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32
img_height = 224
img_width = 224

In [4]:
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 50947 files belonging to 7 classes.


2022-09-19 23:08:07.951550: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 23:08:07.960549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 23:08:07.963205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 23:08:07.969976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [5]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.


In [6]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(training_s, validation_data=validation_s, epochs=5)

Epoch 1/5


2022-09-19 23:09:20.420498: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


12495/12495 [==============================] - 2083s 166ms/step - loss: 1.4162 - accuracy: 0.4550 - val_loss: 1.3241 - val_accuracy: 0.5034
Epoch 2/5
12495/12495 [==============================] - 2065s 165ms/step - loss: 1.3355 - accuracy: 0.4902 - val_loss: 1.2810 - val_accuracy: 0.5155
Epoch 3/5
12495/12495 [==============================] - 2066s 165ms/step - loss: 1.3133 - accuracy: 0.4999 - val_loss: 1.2729 - val_accuracy: 0.5200
Epoch 4/5
12495/12495 [==============================] - 2073s 166ms/step - loss: 1.3027 - accuracy: 0.5043 - val_loss: 1.2599 - val_accuracy: 0.5285
Epoch 5/5
12495/12495 [==============================] - 2073s 166ms/step - loss: 1.2970 - accuracy: 0.5071 - val_loss: 1.2573 - val_accuracy: 0.5298


In [11]:
model.save('0920_EffiB4_frozenfit.h5')

In [12]:
for layer in base_model.layers:
    layer.trainable = True

In [13]:
opt = keras.optimizers.Adam(learning_rate=1e-5)

In [15]:
es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=30, callbacks=[es])

Epoch 1/30
12495/12495 [==============================] - 9379s 749ms/step - loss: 1.0917 - accuracy: 0.5975 - val_loss: 0.7800 - val_accuracy: 0.7087
Epoch 2/30
12495/12495 [==============================] - 9306s 745ms/step - loss: 0.7488 - accuracy: 0.7206 - val_loss: 0.6691 - val_accuracy: 0.7547
Epoch 3/30
 2489/12495 [====>.........................] - ETA: 2:01:53 - loss: 0.6768 - accuracy: 0.7470

In [ ]:
model.save('0920_EffiB4_new.h5')